# Joular and CK data analysis

In [1]:
import requests
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import statistics
import re
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

BASE_URL = "http://localhost:8080/api/v1"

# Exploration of data

- **First objective**: aggregate all the joular data for each method and keep the methods having 30 values
- **Second objective**: remove all the aberrant values like the values that are too high or too low compared to the other values
- **Third objective**: sort all the data in function of their values in descending order
- **Fourth objective**: for a few methods (those having the highest values), compare the distribution of the values with the CK analysis of this method

## Aggregate the joular values and getting the methods with 30 values

In [2]:
def getEndpoint(endpoint, params=None):
    URL = BASE_URL + endpoint
    r = requests.get(url=URL, params=params)
    return r.json()

def getAllJoularDataForMethodsHaving30Values(sha=""):
    allResults = []
    endpoint = "/joular/aggregates"
    if (sha != ""):
        endpoint += "/by-commit/" + sha
    result = getEndpoint(endpoint)
    allResults.extend(result["content"])
    while(not result["last"]):
        page = result["number"] + 1
        result = getEndpoint(endpoint, {"page": page})
        allResults.extend(result["content"])
    return allResults

def getSeveralCkDataForOneMethod(commitSha: str, className: str, methodSignature: str, ckMetrics: list[str]):
    endpoint = "/ck-entities/by-commit-and-ast-elem/" + commitSha
    params = {
        "className": className,
        "methodSignature": methodSignature,
        "names": ckMetrics
    }
    return getEndpoint(endpoint=endpoint, params=params)

def createBoxplotJoular(repositoryName, methodSignature, allValues):
    df = pd.DataFrame({"allValues":allValues})
    fig = px.box(df, y="allValues", title="Joular values for " + methodSignature + " of repository " +repositoryName, points="outliers")
    fig.show()

In [3]:
def removeOutliersByZScore(data, threshold=3):
    zScores = np.abs(stats.zscore(data))
    #zScores = np.abs((data - np.mean(data)) / np.std(data))
    """boolScore = zScores < threshold
    for i in range(len(data)):
        print(str(data[i]) + "   " + str(zScores[i]) + "  " + str(boolScore[i]))"""
    return data[zScores < threshold]

In [4]:
def removeOutliersByIQR(allValues):
    df = pd.DataFrame({"allValues":allValues})
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    """print("Q1 = ", Q1["allValues"])
    print("Q3 = ", Q3["allValues"])
    print("IQR = ", IQR["allValues"])
    print("Lower limit = ", Q1["allValues"] - 1.5 * IQR["allValues"])
    print("Upper limit = ", Q3["allValues"] + 1.5 * IQR["allValues"])"""
    return df[~((df < (Q1 - 1.5 * IQR))|(df > (Q3 + 1.5 * IQR))).any(axis=1)]

### Data for all repositories

In [5]:
allData = getAllJoularDataForMethodsHaving30Values()
print(allData)
joularDataRepo = sorted(allData, key=lambda x: x["allValues"], reverse=True)
joularDataRepo = []
for methodData in joularDataRepo:
    repositoryName = methodData["commit"]["repository"]["name"]
    methodSignature = methodData["measurableElement"]["methodSignature"]
    allValues = methodData["allValues"]
    allValuesAfterStd = removeOutliersByIQR(allValues)
    #print(len(allValuesAfterIQR))
    if (len(allValuesAfterStd) == 30):
        joularDataRepo.append(methodData)

    #createBoxplotJoular(repositoryName, methodSignature, allValues)
    #createBoxplotJoular(repositoryName, methodSignature, allValuesWithoutOutliers)
    #createBoxplotJoular(repositoryName, methodSignature, allValuesAfterIQR["allValues"])
print(len(joularDataRepo))

dataAllRepositories = []
for doc in joularDataRepo:
    dataAllRepositories.extend({"Repository": row["commit"]["repository"]["name"], "Values":val, "Class": row["measurableElement"]["className"]} for val in row["allValues"])
ckDataFrame = pd.DataFrame(dataAllRepositories)

#figRepository = px.violin(ckDataFrame, y="Values", x="Repository", points="all")
#figRepository.show()

[{'allValues': [171.7187, 175.2659, 180.3976, 166.86119, 181.3882, 222.1479, 164.66539, 181.4388, 154.69429, 175.5632, 204.0782, 193.16339, 175.42729, 175.1138, 194.8874, 169.566, 162.4512, 177.38431, 225.8198, 181.7419, 168.6079, 178.34781, 158.9998, 169.99641, 174.72319, 182.36531, 224.55399, 171.2341, 177.8619, 184.2046], 'commit': {'sha': '5c9d8989f968d0ee3a942b411ef7fe121ed94609', 'repository': {'name': 'jabref', 'owner': 'JabRef'}}, 'measurableElement': {'astElem': 'method', 'filePath': '/home/student/j/m/jmaquoi/sentinel/open-source-repositories/jabref/src/main/java/org/jabref/logic/net/URLDownload.java', 'className': 'org.jabref.logic.net.URLDownload', 'methodSignature': 'getMimeType/0', 'variableName': None, 'classType': None}}, {'allValues': [7.9964, 11.0588, 2.6065, 7.7408, 8.4729, 6.6718, 6.4365, 6.2338, 2.1703, 16.131, 5.6093, 8.1704, 6.9321, 5.3873, 8.2181, 10.3354, 16.849, 13.1049, 10.6579, 6.7155, 10.8692, 5.0552, 16.1295, 1.6721, 3.2108, 6.4403, 15.5218, 5.5654, 15.770

### Aggregate by repository and comparison with CK values

In [6]:
def sortData(data):
    return sorted(data, key=lambda x: x["allValues"], reverse=True)

def getTheFirstHighestValues(data, end=5):
    return data[:end]

def createViolinJoular(data, granularity="Repository", points="all"):
    dataAllRepositories = []
    for doc in data:
        dataAllRepositories.extend({"Repository": doc["commit"]["repository"]["name"], "Values":val, "Class": doc["measurableElement"]["className"]} for val in doc["allValues"])
    df = pd.DataFrame(dataAllRepositories)

    figRepository = px.violin(df, y="Values", x=granularity, points=points)
    figRepository.show()

def createMultipleBoxplot(dataOneRepo):
    dataframePreparation = []
    for method in dataOneRepo:
        dataframePreparation.append({"Method": method["measurableElement"]["methodSignature"], "Values":method["allValues"], "Class":method["measurableElement"]["className"]})
    df = pd.DataFrame(dataframePreparation)
    print(df)
    fig = px.box(df, x="Method", y="Values", points="outliers")
    fig.show()
        


def createBarChart(dataframe, xAxisName, yAxisName, title):
    fig = px.bar(dataframe, x=xAxisName, y=yAxisName, title=title)
    fig.update_layout(yaxis_range=[0, 40])
    return fig

def prepareCkDataFrame(ckValues):
    values = {item["name"]:item["value"] for item in ckValues}
    return pd.DataFrame(list(values.items()), columns=["Metric", "Value"])

def createBoxplot(dataframe, title):
    return px.box(dataframe, y="value", title=title, points="all")

def prepareJoularDataFrame(joularValues):
    return pd.DataFrame({"value": joularValues})

def createSubplot(fig1, fig2, title):
    fig1Traces = []
    fig2Traces = []
    for trace in range(len(fig1["data"])):
        fig1Traces.append(fig1["data"][trace])
    for trace in range(len(fig2["data"])):
        fig2Traces.append(fig2["data"][trace])

    
    figure = make_subplots(rows=1, cols=2, subplot_titles=("Joular values", "Ck values"))
    for traces in fig1Traces:
        figure.append_trace(traces, row=1, col=1)
    for traces in fig2Traces:
        figure.append_trace(traces, row=1, col=2)

    figure.update_layout(title_text=title)
    figure.show()

## Methods to get specific fields from data from db

In [7]:
MEASURABLE_ELEMENT = "measurableElement"
CLASS_NAME = "className"
METHOD_SIGNATURE = "methodSignature"
ALL_VALUES = "allValues"
COMMIT = "commit"
REPOSITORY = "repository"
REPO_NAME = "name"

In [8]:
# Repo data
def getRepositoryName(method):
    return method[COMMIT][REPOSITORY][REPO_NAME]

# Joular data
def getClassName(method):
    return method[MEASURABLE_ELEMENT][CLASS_NAME]

def getMethodSignature(method):
    return method[MEASURABLE_ELEMENT][METHOD_SIGNATURE]

def getAllJoularValues(method):
    return method[ALL_VALUES]

def getMeanOfAllJoularValues(method):
    return statistics.mean(getAllJoularValues(method))


# CK data
def getMetricName(ckData):
    return ckData["name"]

def getMetricValue(ckData):
    return ckData["value"]

## Creation of the figures

Const used as key for the joular and ck dataframe

In [9]:
METHOD_DF = "Method"
CLASS_DF = "Class"
JOULAR_VALUES_DF = "JoularValues"
MEAN_JOULAR_VALUES_DF = "MeanJoularValue"
REPO_DF = "Repository"

In [10]:
def matchOnlyArgsNames(matchedArgs):
    argsNames = []
    if len(matchedArgs) > 0:
        for matchedArg in matchedArgs[0].split(','):
            argName = re.sub(r'<.*?>', '', matchedArg).split('.')[-1]
            argsNames.append(argName)
    return argsNames

In [11]:
def simplifyMethodName(methodName):
    matches = re.findall(r'\[([^\]]+)', methodName)
    if len(matches) == 0:
        return methodName.split('/')[0] + '/'
    simplifiedArgs = matchOnlyArgsNames(matches)
    simplifiedName = methodName.split('/')[0] + '/' + str(len(simplifiedArgs)) + '[' + ','.join(simplifiedArgs) + ']'
    return simplifiedName

In [12]:
def getNamesXAxis(row):
    nameLegend = simplifyMethodName(row["Method"])
    return nameLegend + " " + row["Class"].split(".")[-1][:20]

In [13]:
def createBoxplotBarchartFigure(df, ckMetric, repository):    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for index, row in df.iterrows():
        showLegend = False
        if index == 1:
            showLegend = True
        nameBar = getNamesXAxis(row)

        # Box plot
        boxTrace = go.Box(
            #x=[getNamesXAxis(row)],
            y=row[JOULAR_VALUES_DF],
            name=nameBar,
            marker=dict(color='rgb(252,141,98)'),
            legendgroup="boxplots",
            showlegend=showLegend
        )
        fig.add_trace(boxTrace, secondary_y=True)

        # Bar chart
        xBarChart = [nameBar]
        yBarChart = [row[ckMetric]]
        traceBarchart = go.Bar(x=xBarChart, y=yBarChart, name=ckMetric, marker=dict(color='rgb(141,160,203)'), legendgroup="barcharts", showlegend=showLegend)
        fig.add_trace(traceBarchart)

    # Trendline for the bar chart
    overallTrendline = np.polyfit(np.arange(len(df)), df[ckMetric], 1)
    overallSlope = overallTrendline[0]
    overallTrendlineValues = np.polyval(overallTrendline, np.arange(len(df)))
    traceTrendline = go.Scatter(x=[f"{getNamesXAxis(row)}" for _, row in df.iterrows()], y=overallTrendlineValues, name=f'Trendline for {ckMetric} (Slope: {overallSlope:.2f})', mode="lines", line=dict(color='red'))
    #fig.add_trace(traceTrendline)
        
    fig['layout'].update(legend=dict(traceorder='normal'))
    fig['layout'].update(title= "Comparison of joular data and '" + ckMetric + "' metric for repository " + repository, height=1000)
    fig.show()

In [14]:
def createScatterPlot(df, ckMetric, repoName):
    fig = go.Figure()
    scatterTrace = go.Scatter(
        x=df[ckMetric],
        y=df[MEAN_JOULAR_VALUES_DF],
        mode="markers",
        text=[f"{getNamesXAxis(row)}" for _, row in df.iterrows()]
    )
    fig.add_trace(scatterTrace)

    # Update layout
    fig.update_layout(
        title=f"{ckMetric} and mean of Joular values, for each method of repository \"{repoName}\"",
        xaxis_title=ckMetric,
        yaxis_title="Mean of Joular Values"
    )

    fig.show()

In [33]:
def createTsne(df, coloredFeature, perplexity):
    #features = ['MeanJoularValue', 'cbo', 'rfc', 'loc']
    df['constructor'] = df['constructor'].astype(int)
    df['hasJavaDoc'] = df['hasJavaDoc'].astype(int)
    numericColumns = df.select_dtypes(include=['number']).columns
    X = df[numericColumns]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    X_tsne = tsne.fit_transform(X_scaled)

    hover_data = {'Method': df[METHOD_DF], 'Class': df[CLASS_DF], 'Repository': df[REPO_DF]}

    fig = px.scatter(
        X_tsne, x=0, y=1,
        hover_data=hover_data,
        color=df[coloredFeature], labels={'color':coloredFeature}
    )
    fig.show()

## Creation of the dataframe containing all data for all repositories

### Utils functions

In [16]:
def exportDataframeToFile(df, path):
    df.to_csv(path + "/allData25Values.csv", index=False)

In [17]:
def addRepoDataframeToAllDataframe(allDf, dfToAdd):
    frames = [allDf, dfToAdd]
    return pd.concat(frames)

#### Init of dataframe

In [18]:
def createCkDict(methodData, mergedData, sha, allowedCkMetrics):
    className = getClassName(methodData)
    methodSignature = getMethodSignature(methodData)

    ckDataMethod = getSeveralCkDataForOneMethod(commitSha=sha, className=className, methodSignature=methodSignature, ckMetrics=allowedCkMetrics)
    
    for metricData in ckDataMethod:
        mergedData[getMetricName(metricData)].append(getMetricValue(metricData))
    return mergedData

In [19]:
def createJoularDict(methodData, mergedData):
    mergedData[REPO_DF].append(getRepositoryName(methodData))
    mergedData[CLASS_DF].append(getClassName(methodData))
    mergedData[METHOD_DF].append(simplifyMethodName(getMethodSignature(methodData)))
    mergedData[JOULAR_VALUES_DF].append(getAllJoularValues(methodData))
    mergedData[MEAN_JOULAR_VALUES_DF].append(getMeanOfAllJoularValues(methodData))
    return mergedData

In [20]:
def initDataframe(allowedCkMetrics):
    dataframe = {
        REPO_DF: [],
        CLASS_DF : [],
        METHOD_DF : [],
        JOULAR_VALUES_DF : [],
        MEAN_JOULAR_VALUES_DF: []
    }
    for ckMetric in allowedCkMetrics:
        dataframe[ckMetric] = []
    return dataframe

#### Outliers

In [21]:
def removeOutliersByStd(allValues):
    mean = np.mean(allValues)
    stdDev = np.std(allValues)
    """print(mean, stdDev, "\n")
    for x in allValues:
        print(x, np.abs(mean - x), 3*stdDev)"""
    return [x for x in allValues if (np.abs(mean - x) <= 3 * stdDev)]

In [22]:
def removeOutliers(data):
    print("Len with outliers : ", len(data))
    only25ValuesAndMore = []
    for methodData in data:
        allValues = methodData["allValues"]
        allValuesAfterStd = removeOutliersByStd(allValues)
        if (len(allValuesAfterStd) >= 25):
            methodData["allValues"] = allValuesAfterStd
            only25ValuesAndMore.append(methodData)
    print("Len without outliers (25+ values) : ", len(only25ValuesAndMore))
    return only25ValuesAndMore

### Dataframe for one repo

In [23]:
def createJoularAndCkDataframeForOneRepo(dataOneRepo, sha, allowedCkMetrics):
    mergedData = initDataframe(allowedCkMetrics)
    for method in dataOneRepo:
        mergedData = createJoularDict(method, mergedData)
        mergedData = createCkDict(method, mergedData, sha, allowedCkMetrics)
    return pd.DataFrame(mergedData)

In [24]:
def createOneRepoDataframe(sha, allowedCkMetrics):
    joularDataRepo = getAllJoularDataForMethodsHaving30Values(sha)
    sortedJoularDataRepo = sortData(joularDataRepo)
    joularDataRepoWithoutOutliers = removeOutliers(sortedJoularDataRepo)
    df = createJoularAndCkDataframeForOneRepo(joularDataRepoWithoutOutliers, sha, allowedCkMetrics)
    return df

### One dataframe for all repos

In [25]:
def createAllRepoDataframe(allCommitSha, allowedCkMetrics):
    allRepoDataDf = pd.DataFrame()
    for sha in allCommitSha:
        df = createOneRepoDataframe(sha, allowedCkMetrics)
        allRepoDataDf = addRepoDataframeToAllDataframe(allRepoDataDf, df)
    return allRepoDataDf

In [26]:
def createScatterPlotForEachRepo(allRepoDataDf, allowedCkMetrics):
    repositories = allRepoDataDf[REPO_DF].unique()
    for repo in repositories:
        repoDf = allRepoDataDf[allRepoDataDf[REPO_DF] == repo]
        for metric in allowedCkMetrics:
            createScatterPlot(repoDf, metric, repo)

In [27]:
def createBoxplotBarchartForEachRepo(allRepoDataDf, allowedCkMetrics):
    repositories = allRepoDataDf[REPO_DF].unique()
    for repo in repositories:
        repoDf = allRepoDataDf[allRepoDataDf[REPO_DF] == repo]
        for metric in allowedCkMetrics:
            createBoxplotBarchartFigure(repoDf, metric, repo)

## Called functions

In [28]:
allCommitSha = ["066f4cf207359e06d30911a553dedd054aef595c", "5c9d8989f968d0ee3a942b411ef7fe121ed94609", "12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "3ed1f1a064a10e53adc2ad8c0b46a4b2c148ee21", "59e5152722198526c6ffe5361de7d1a6a87275c7"]
#allCommitSha = ["066f4cf207359e06d30911a553dedd054aef595c"]
#allowedCkMetrics = ["cbo", "rfc", "loc"]
allowedCkMetrics = ["constructor", "cbo", "cboModified", "fanin", "fanout", "wmc", "rfc", "loc", "returnsQty", "variablesQty", "parametersQty", "methodsInvokedQty", "methodsInvokedLocalQty", "methodsInvokedIndirectLocalQty", "loopQty", "comparisonsQty", "tryCatchQty", "parenthesizedExpsQty", "stringLiteralsQty", "numbersQty", "assignmentsQty", "mathOperationsQty", "maxNestedBlocksQty", "anonymousClassesQty", "innerClassesQty", "lambdasQty", "uniqueWordsQty", "modifiers", "logStatementsQty", "hasJavaDoc"]

allRepoDf = createAllRepoDataframe(allCommitSha, allowedCkMetrics)
print(len(allRepoDf))
#exportDataframeToFile(allRepoDf, "/home/jerome/Documents/Assistant/Recherche/joular-scripts")

Len with outliers :  197
Len without outliers (25+ values) :  197
Len with outliers :  76
Len without outliers (25+ values) :  76
Len with outliers :  73
Len without outliers (25+ values) :  73
Len with outliers :  177
Len without outliers (25+ values) :  177
Len with outliers :  15
Len without outliers (25+ values) :  15
538


In [59]:
allowedCkMetrics = ["cbo", "rfc", "loc"]
createScatterPlotForEachRepo(allRepoDf, allowedCkMetrics)

In [60]:
createBoxplotBarchartForEachRepo(allRepoDf, allowedCkMetrics)

In [35]:
perplexities = [5, 10, 20, 30, 40, 50]
for i in perplexities:
    createTsne(allRepoDf, 'MeanJoularValue', i)
#createTsne(allRepoDf, 'rfc')
#createTsne(allRepoDf, 'cbo')
#createTsne(allRepoDf, 'MeanJoularValue')

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/jerome/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Data with aberrant negative values

The method *verifyCollection* has the value -530894.

In [12]:
createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.orm.test.bootstrap.registry.classloading.PhantomReferenceLeakDetector", "verifyCollection/3[java.lang.ref.ReferenceQueue<T>,int,int]")

The method *accept* below has the value -217371.5

In [13]:
createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.tool.schema.internal.exec.GenerationTargetToDatabase", "accept/1[java.lang.String]")